In [1]:
# 셀레니움
from selenium import webdriver 
# 셀레니움 By 클래스 가져오기
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
import time
import random
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
# NoSuchElementException import 추가
from selenium.common.exceptions import NoSuchElementException  
import pandas as pd
from selenium.common.exceptions import StaleElementReferenceException
import logging
from selenium.common.exceptions import TimeoutException, WebDriverException
from datetime import datetime
import re

In [2]:
# 크롬 웹 드라이버 초기화 및 카카오 맵 페이지로 이동
driver = webdriver.Chrome() 
url = 'https://m.map.kakao.com/'
driver.get(url)

# 검색창 element와 검색창 id 가져오기
search_box = driver.find_element(By.ID,'innerQuery')

# 키워드 입력
# 키워드 바꿔주기
search_box.send_keys("마포구 빵")

# Enter 키 누르기
search_box.send_keys(Keys.RETURN)

# 페이지가 로드될 때까지 잠시 대기
time.sleep(3)

# 베너 닫기
driver.find_element(By.XPATH, f'//*[@id="daumWrap"]/div[1]/div/a[2]/span').click()

# 더보기 버튼을 누를 수 있는지 여부 확인하는 함수
def can_click_load_more_button():
    try:
        load_more_button = driver.find_element(By.XPATH, '//*[@id="daumContent"]/div[4]/div[2]/div[3]/a')  # 더보기 버튼의 XPATH
        return load_more_button.is_enabled() and load_more_button.is_displayed()
    except:
        return False

# 더보기 버튼 클릭 함수
def click_load_more_button():
    load_more_button = driver.find_element(By.XPATH, '//*[@id="daumContent"]/div[4]/div[2]/div[3]/a')  # 더보기 버튼의 XPATH
    load_more_button.click()
    time.sleep(3)  # 버튼을 클릭한 후에 충분한 시간 대기 (페이지가 로딩될 때까지)

# 더보기 버튼을 누를 수 없을 때까지 반복해서 클릭
while can_click_load_more_button():
    click_load_more_button()
    
# 페이지에 표시된 정보의 갯수 확인
all_results = driver.find_elements(By.XPATH, '//*[@id="placeList"]/li')
print("페이지에 표시된 정보의 갯수:", len(all_results))

driver.execute_script("window.scrollTo(0, -document.body.scrollHeight);")

# 스크롤하여 요소가 보이도록 만드는 함수
def scroll_to_element(element):
    driver.execute_script("arguments[0].scrollIntoView(true);", element)
    
# 후기 더보기 누르기, 후기 접기는 정지
def class_click():
    prev_class = 'link_more'
    while True:
        element = driver.find_element(By.XPATH, f'//*[@id="mArticle"]/div[*]/div/div[2]/a')
        element_class = element.get_attribute("class")
        # 이전 클래스와 현재 클래스가 다른지 확인
        if prev_class != element_class:
            print("Class changed. Stopping click.")
            break
        # 클릭하고 잠시 대기
        element.click()
        time.sleep(wait)
        # 현재 클래스를 이전 클래스에 저장
        prev_class = element_class

# find_total 수정       
def find_data():
    g = 1  # 루프 밖에서 변수 초기화
    max_attempts = 3  # 최대 시도 횟수를 설정하여 무한 루프 방지
    attempts = 0
    while attempts < max_attempts:
        try:
            xpath_id = f'//*[@id="mArticle"]/div[*]/div/div[2]/ul/li[{g}]/div[1]/div[1]/div[1]/a/span'
            xpath_date = f'//*[@id="mArticle"]/div[*]/div/div[2]/ul/li[{g}]/div[1]/div[1]/div[2]/span[6]'
            xpath_rv = f'//*[@id="mArticle"]/div[*]/div/div[2]/ul/li[{g}]/div[1]/div[*]/p/span'
            
            now = datetime.now().strftime('%Y-%m-%d - %H:%M:%S')
            time_list.append(now) 
            
            try:
                id_element = WebDriverWait(driver, wait).until(EC.visibility_of_element_located((By.XPATH, xpath_id)))
                temp_ID = id_element.text.strip()
                if temp_ID:
                    ID.append(id_element.text)
                else:
                    ID.append('정보 없음')
            except (WebDriverException or NoSuchElementException or TimeoutException): 
                ID.append('정보 없음')
            time.sleep(wait)

            try:
                date_element = WebDriverWait(driver, wait).until(EC.visibility_of_element_located((By.XPATH, xpath_date)))
                Date.append(date_element.text)
            except (WebDriverException or NoSuchElementException or TimeoutException): 
                break

            time.sleep(wait)

            try:
                rv_element = WebDriverWait(driver, wait).until(EC.visibility_of_element_located((By.XPATH, xpath_rv)))
                temp_review_text = rv_element.text.strip()
                if temp_review_text:
                    Review_text.append(temp_review_text)
                else:
                    Review_text.append('정보 없음')
            except (WebDriverException or NoSuchElementException or TimeoutException):  
                Review_text.append('정보 없음')
            
            
            driver.execute_script("window.scrollBy(0, 200);")
            g += 1
            attempts = 0
        # 이 부분은 모든 요소에 대한 크롤링 시도가 실패했을 때를 처리
        except (WebDriverException or NoSuchElementException or TimeoutException):  
            now = datetime.now().strftime('%Y-%m-%d - %H:%M:%S')
            time_list.append(now)
            attempts += 1
            
        if attempts >= max_attempts:
            print("최대 시도 횟수에 도달하였거나 더 이상 크롤링할 요소가 없습니다.")
            break
        
# 크롤링 시작
store = []
review_score = []
review_count = []
address = []
ID = []
Review_text = []
Date = []
time_list = []
number = []

wait = random.uniform(1, 5)
# t_ele = driver.find_elements(By.CLASS_NAME, "link_result") 
t_ele = driver.find_elements(By.CLASS_NAME, "tit_g")
# t_ele = driver.find_elements(By.XPATH, f'//*[@id="placeList"]/li[{i}]/a[1]/span[2]/span[1]/strong') 
for i in range(len(all_results) + 1): 
    # 배너 건너뛰기
    if t_ele[i].get_attribute("class") != 'tit_g':
        print('banner!')
        continue
    else:
        t_ele[i].click()
        time.sleep(wait)
    # 가게 이름, 주소, 후기 갯수, 후기 점수 크롤링
        try:
            # 가게 이름 크롤링
            cw = driver.find_element(By.XPATH, '//*[@id="mArticle"]/div[1]/div/div[1]/h2').text
            # 후기 갯수 크롤링
            qa = driver.find_element(By.XPATH, '//*[@id="mArticle"]/div[1]/div/div[1]/div[1]/a[1]/span[2]/span[1]').text
            # 주소 크롤링
            ad = driver.find_element(By.CLASS_NAME, 'txt_address').text
            # 점수 크롤링
            ty = driver.find_element(By.XPATH, '//*[@id="mArticle"]/div[1]/div/div[1]/div[1]/a[1]/span[1]/span[1]').text
            time.sleep(wait)

        
            # 후기 갯수만큼 가게 이름, 주소, 후기 갯수, 후기 점수, 가게 넘버 늘리기
            temp_store = [cw for _ in range(int(qa))]
            for h in temp_store:
                store.append(h)
                
            temp_rv = [int(qa) for _ in range(int(qa))]
            for k in temp_rv:
                review_count.append(k)
                
            temp_add = [ad for _ in range(int(qa))]
            for j in temp_add:
                address.append(j)
                
            temp_score = [ty for _ in range(int(qa))]
            for l in temp_score:
                review_score.append(l)
            temp_num = [i for _ in range(int(qa))]
            for p in temp_num:
                number.append(p)
                
            # time.sleep(wait)
            # 후기 부분까지 스크롤 내릴 위치 찾기
            more_reviews_link = driver.find_element(By.CLASS_NAME, 'rate_intro')
            # 후기 더보기 링크가 보이도록 스크롤
            scroll_to_element(more_reviews_link)   
            # 후기 전체 페이지로 넘어가는 클릭
            driver.find_element(By.XPATH, f'//*[@id="mArticle"]/div[*]/div/ul[*]/li[1]/div[1]/div[2]/p').click()
            time.sleep(wait)
            
            # 첫 번째 더보기 버튼 클릭
            try:
                class_click()
            except NoSuchElementException:
                print("Can't Click!")
                pass
            time.sleep(wait)

            # 후기 크롤링 시작   
            find_data()
            ID.pop()
            
            # # 검색 결과 페이지로 돌아가기
            driver.back()
            time.sleep(1)
            driver.back()
            
            # 페이지가 로드될 때까지 잠시 대기
            time.sleep(wait)   
            
            # 후기 없는 경우 뒤로가기
        except NoSuchElementException:
            driver.back()
            continue

        driver.execute_script("window.scrollTo(0, -document.body.scrollHeight);")
    
# DF에 저장하기
kakao = pd.DataFrame()

# '구'까지 슬라이싱해서 'addr' 칼럼에 추가하는 함수
def slice_to_gu(address):
    gu_index = address.find('구') + 1
    if gu_index:
        return address[:gu_index]
    else:
        return address

kakao['Platform'] = ['Kakao' for _ in range(len(all_results)-1)]
kakao['Number'] = number
kakao['Store'] = store
kakao['Address'] = address
kakao['Addr'] = kakao['Address'].apply(lambda x: slice_to_gu(x))
kakao['Review_score'] = review_score
kakao['Review_counts'] = review_count
kakao['ID'] = ID
kakao['Date'] = Date
kakao['Review_text'] = Review_text
kakao['Time'] = time_list

kakao.to_csv('kakao.csv')

kakao

페이지에 표시된 정보의 갯수: 76
Class changed. Stopping click.


StaleElementReferenceException: Message: stale element reference: stale element not found
  (Session info: chrome=123.0.6312.106); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#stale-element-reference-exception
Stacktrace:
	GetHandleVerifier [0x00007FF7F1A57072+63090]
	(No symbol) [0x00007FF7F19C2CC2]
	(No symbol) [0x00007FF7F185EC65]
	(No symbol) [0x00007FF7F186E78B]
	(No symbol) [0x00007FF7F186EFEA]
	(No symbol) [0x00007FF7F1864B29]
	(No symbol) [0x00007FF7F186F028]
	(No symbol) [0x00007FF7F1864B29]
	(No symbol) [0x00007FF7F1862D3A]
	(No symbol) [0x00007FF7F1866462]
	(No symbol) [0x00007FF7F18E4062]
	(No symbol) [0x00007FF7F18C6FDA]
	(No symbol) [0x00007FF7F18E3412]
	(No symbol) [0x00007FF7F18C6D83]
	(No symbol) [0x00007FF7F18983A8]
	(No symbol) [0x00007FF7F1899441]
	GetHandleVerifier [0x00007FF7F1E525CD+4238285]
	GetHandleVerifier [0x00007FF7F1E8F72D+4488493]
	GetHandleVerifier [0x00007FF7F1E87A0F+4456463]
	Get

In [4]:
print(len(store))
print(len(review_count))
print(len(review_score))
print(len(address))
print(len(number))
print(len(ID))
print(len(Date))
print(len(Review_text))
print(len(time_list))

# 모두 갯수가 같아야함

273
273
273
273
273
42
42
41
42
